### Creazione grafo
nodi totali: 3110737 \
nodi che presentano l'anno: 3110687

In [ ]:
import networkx as nx
import pandas as pd
import re
from queue import Queue
from collections import defaultdict
from itertools import combinations

#data = pd.read_csv("imdb-actors-actresses-movies.tsv", sep="\t")
data = pd.read_csv("prova.tsv", sep="\t")
edgesDict = data.to_dict("index")
idActorDict = dict(enumerate(data["Actor"].unique()))
actorIdDict = {v: k for k ,v in idActorDict.items()}
idMovieDict = dict(enumerate(data["Movie"].unique(), start=len(idActorDict)))
movieIdDict = {v: k for k, v in idMovieDict.items()}

def getYear(movieString):
    match = re.search(".*\(.*(\d\d\d\d).*\)", movieString)
    if match is None:
        year = "Not found"
    else:
        yearString = match.group(1)
        year = int(yearString)
    return movieString, year

movieYearDict= dict(map(getYear,data["Movie"].unique()))

In [2]:
G = nx.Graph()
G.add_nodes_from([(id, {"name": a, "type": 0}) for id, a in idActorDict.items()])
G.add_nodes_from([(id, {"name": m, "type": 1, "year": movieYearDict[m]}) for id, m in idMovieDict.items()])
G.add_edges_from([(actorIdDict[v["Actor"]], movieIdDict[v["Movie"]]) for k, v in edgesDict.items()])

### Es 1.E
Vengono creati 2 dizionari, le cui istanze sono: Nome dell'attore-Chiave e Anno film (minimo o massimo)-Valore.
Un ciclo for poi gira su tutti i film che hanno come data di produzione un tempo compreso fra *min_year* e *maxYear*, e guarda gli attori che hanno partecipato a tali film. Per tali attori aggiorna il valore nel dizionario se l'anno del film è minore del valore presente nel dizionario o crea la voce se esso non è presente.
Il secondo ciclo for fa la stessa cosa però con i tempi massimi.
Infine il terzo ciclo calcola i tempi di attività nel periodo *min_year* e *maxYear* e trova l'attore che ha lavorato di più.

In [ ]:
def computeLongevousActor(maxYear):
  actorMinYearDict = {}
  actorMaxYearDict = {}
  for node in G.nodes().data():
      if (node[1]["type"] == 1 and node[1]["year"] != "Not found" and node[1]["year"] <= maxYear):
          for l in G.neighbors(node[0]):
              if(l not in actorMinYearDict or node[1]["year"] < actorMinYearDict.get(l)):
                  actorMinYearDict.update({l : node[1]["year"]})
  for node in G.nodes().data():
      if (node[1]["type"] == 1 and node[1]["year"] != "Not found" and node[1]["year"] <= maxYear):
          for l in G.neighbors(node[0]):
              if(l not in actorMaxYearDict or node[1]["year"] > actorMaxYearDict.get(l)):
                  actorMaxYearDict.update({l : node[1]["year"]})
  maxWorkPeriod = 0
  actorMax = None
  for a in actorMinYearDict.items():
      if maxWorkPeriod < (actorMaxYearDict.get(a[0]) - actorMinYearDict.get(a[0])):
          actorMax = a[0]
          maxWorkPeriod = actorMaxYearDict.get(a[0]) - actorMinYearDict.get(a[0])
  
  print("L'attore che ha lavorato di più fino al "+str(maxYear)+" è: ")
  print(str(idActorDict[actorMax])+" con un periodo di attività di: "+str(maxWorkPeriod)+" anni")
  print()

def computeAllLongevousActor():
    for x in range (1930, 2030, 10):
        computeLongevousActor(x)

### Es 2.1

In [ ]:
def generateSubGraphGCC(G, maxYear):
    SG = G.subgraph([n for n, a in G.nodes().data() if a["type"] == 0 or (a["year"] != "Not found" and a["year"] < maxYear)])
    tempLCC = max(nx.connected_components(SG), key=len)
    #tempLCC = sorted(nx.connected_components(SG), key=len, reverse=True)[0]
    print("a")
    LCC = SG.subgraph(tempLCC)
    degreeDict = dict(LCC.degree())
    m = max(degreeDict.values())
    #startNode = [k for k, v in degreeDict.items() if v == m][0]
    startNode = max(LCC.degree,key=lambda x: x[1])[0]
    print(startNode)
    Bu = customBFS(LCC, startNode)
    computeDiameter(LCC, Bu)

def customBFS(LCC, startNode):
    visited = {}
    queue = Queue()
    queue.put(startNode)
    visited[startNode] = 0
    while not queue.empty():
        currentNode = queue.get()
        for nextNode in LCC.neighbors(currentNode):
            if nextNode not in visited:
                queue.put(nextNode)
                visited[nextNode]=visited[currentNode]+1
    Bu = defaultdict(list)
    for key, value in visited.items():
        Bu[value].append(key)
    return Bu

def computeDiameter(LCC, Bu):
    i = lb = max(Bu)
    ub = 2*lb
    while ub > lb:
        eccDict = nx.eccentricity(LCC, Bu[i])
        Bi = max(eccDict.values())
        maxVal = max(Bi,lb)
        if maxVal > 2*(i - 1):
            print("diametro: ", maxVal)
            return maxVal
        else: 
            lb = maxVal
            ub = 2*(i - 1) 
        i = i - 1 
    print("diametro: ", lb)
    return lb

generateSubGraphGCC(G, 1950)

""" for x in range (1930, 2030, 10):
    generateSubGraphGCC(G, x) """


### Es 3.I

In [ ]:
def maxCollab(G):
    collaborations = {}
    for n, a in G.nodes().data():
        if a["type"] == 0:
            c = 0
            for nb in G.neighbors(n):
                c += (len(G.adj[nb])-1)
            collaborations.update({a["name"]: c})
    m = max(list(collaborations.values()))
    print(m)
    print([k for k, v in collaborations.items() if v == m][0])
    
maxCollab(G)

### Es 4

In [ ]:
#cella di prova
for k1, v1 in edgesDict.copy().items():
    for k2, v2 in edgesDict.copy().items():
        if v2["Movie"] == v1["Movie"] and v1["Actor"] != v2["Actor"]:
            A.add_edge(v1["Actor"], v2["Actor"])
            edgesDict.pop(k2)
    edgesDict.pop(k1)

    #A.add_edges_from([(v1["Actor"], v2["Actor"]) for k2, v2 in edgesDict.items() if v1["Actor"] != v2["Actor"] and v2["Movie"] == v1["Movie"]])

print(A.nodes.data())
print(A.edges.data())

In [ ]:
A = nx.Graph()
for i in idMovieDict:
    actors = list(G.neighbors(i))
    for j in combinations(actors, 2):
        if A.has_edge(*j):
            A[j[0]][j[1]]['weight'] += 1
        else:
            A.add_edge(j[0],j[1], weight = 1)

print(A.nodes.data())
print(A.edges.data())

In [ ]:
n1, n2, w = max(A.edges().data("weight"), key = lambda x: x[2])
print((idActorDict[n1], idActorDict[n2]))
print(w)

### celle per prove

In [ ]:
diz = {0: "a 1900 (aaa2000/I)", 1: "b 1920"}
x = re.search(".*\(.*(\d\d\d\d).*\)", diz[0]).group(1)
print(type(x))
for k, v in diz.items():
    print(k)
    print(v)